#Installs & HF token

In [1]:
#@title PIP Install
%%capture
!pip -q install --upgrade pip
!pip -q install git+https://github.com/huggingface/diffusers.git
#!pip -q install transformers accelerate safetensors huggingface_hub einops
!pip install -U transformers accelerate peft safetensors datasets

#!pip -q install -U "diffusers>=0.30.0" "peft>=0.12.0" transformers accelerate safetensors pillow torchvision

In [2]:
#@title HF token
from huggingface_hub import login
import os

TOKTOK='' #@param {type: "string"}

HF_TOKEN = os.getenv('HF_TOKEN', TOKTOK)  # set env var or paste interactively
if HF_TOKEN:
    login(token=HF_TOKEN)
else:
    login()

FLUX.1-Kontext LoRA + простой region_token из маски (только локализация)

Пары (context -> target), маска -> один region_token (обучаемая проекция)

Пайплайн только для инференса/семплов; тренируем компоненты (LoRA через PEFT)

Flow-matching (упрощённый таргет), 2×2 packing токенов как в Kontext

##Preparing

In [3]:
#@title Load Libs
import os, math
from pathlib import Path
from typing import List
import torch
import torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.transforms.functional import resize, InterpolationMode, pil_to_tensor, to_pil_image
from PIL import Image
import numpy as np

import cv2
from transformers import DPTForDepthEstimation, DPTImageProcessor

from safetensors.torch import save_file, load_file

from diffusers import FluxKontextPipeline, FlowMatchEulerDiscreteScheduler, logging  # Flux Kontext + flow-matching scheduler
from peft import LoraConfig  # PEFT LoRA адаптеры

from IPython.display import clear_output
from tqdm.notebook import tqdm


logging.set_verbosity_error()


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype  = torch.bfloat16 if torch.cuda.is_available() else torch.float32

Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.


In [4]:
#@title Setups

DATA_ROOT   = "data_kontext"    #@param {type: "string"}
BATCH_SIZE  = 1                 #@param {type: "integer"}
RES         = 512              #@param {type: "integer"}
LR          = 2e-4              #@param {type: "number"}
EPOCHS      = 2                 #@param {type: "integer"}
SAMPLE_EVERY= 2               #@param {type: "integer"}

In [5]:
#@title Upload imgs


from google.colab import files
import os
from pathlib import Path

def upload_imgs(path):
    print(f'Upload to:  {path}')
    os.makedirs(path, exist_ok=True)
    uploaded = files.upload()
    for filename, content in uploaded.items():
        with open(path +'/'+ filename, "wb") as f:
            f.write(content)
    print(f'{path}:  DONE')


upload_imgs(DATA_ROOT + "/red")
upload_imgs(DATA_ROOT + "/context")
upload_imgs(DATA_ROOT + "/target")

os.makedirs(DATA_ROOT +"/depth", exist_ok=True)
os.makedirs(DATA_ROOT + "/masks", exist_ok=True)


Upload to:  data_kontext/red


Saving im (1).png to im (1).png
Saving im (2).png to im (2).png
Saving im (3).png to im (3).png
Saving im (4).png to im (4).png
Saving im (5).png to im (5).png
Saving im (6).png to im (6).png
Saving im (7).png to im (7).png
Saving im (8).png to im (8).png
Saving im (9).png to im (9).png
Saving im (10).png to im (10).png
Saving im (11).png to im (11).png
Saving im (12).png to im (12).png
Saving im (13).png to im (13).png
Saving im (14).png to im (14).png
Saving im (15).png to im (15).png
Saving im (16).png to im (16).png
Saving im (17).png to im (17).png
Saving im (18).png to im (18).png
Saving im (19).png to im (19).png
Saving im (20).png to im (20).png
data_kontext/red:  DONE
Upload to:  data_kontext/context


Saving im (1).png to im (1) (1).png
Saving im (2).png to im (2) (1).png
Saving im (3).png to im (3) (1).png
Saving im (4).png to im (4) (1).png
Saving im (5).png to im (5) (1).png
Saving im (6).png to im (6) (1).png
Saving im (7).png to im (7) (1).png
Saving im (8).png to im (8) (1).png
Saving im (9).png to im (9) (1).png
Saving im (10).png to im (10) (1).png
Saving im (11).png to im (11) (1).png
Saving im (12).png to im (12) (1).png
Saving im (13).png to im (13) (1).png
Saving im (14).png to im (14) (1).png
Saving im (15).png to im (15) (1).png
Saving im (16).png to im (16) (1).png
Saving im (17).png to im (17) (1).png
Saving im (18).png to im (18) (1).png
Saving im (19).png to im (19) (1).png
Saving im (20).png to im (20) (1).png
data_kontext/context:  DONE
Upload to:  data_kontext/target


Saving im (1).png to im (1) (2).png
Saving im (2).png to im (2) (2).png
Saving im (3).png to im (3) (2).png
Saving im (4).png to im (4) (2).png
Saving im (5).png to im (5) (2).png
Saving im (6).png to im (6) (2).png
Saving im (7).png to im (7) (2).png
Saving im (8).png to im (8) (2).png
Saving im (9).png to im (9) (2).png
Saving im (10).png to im (10) (2).png
Saving im (11).png to im (11) (2).png
Saving im (12).png to im (12) (2).png
Saving im (13).png to im (13) (2).png
Saving im (14).png to im (14) (2).png
Saving im (15).png to im (15) (2).png
Saving im (16).png to im (16) (2).png
Saving im (17).png to im (17) (2).png
Saving im (18).png to im (18) (2).png
Saving im (19).png to im (19) (2).png
Saving im (20).png to im (20) (2).png
data_kontext/target:  DONE


In [6]:
#@title Make masks from red

input_dir = Path(DATA_ROOT + "/red")
output_dir = Path(DATA_ROOT + "/masks")

# Define the range for bright red color in HSV
# Red color wraps around the 0/180 mark in HSV, so we need two ranges
lower_red_1 = np.array([0, 100, 100])
upper_red_1 = np.array([10, 255, 255])

lower_red_2 = np.array([160, 100, 100])
upper_red_2 = np.array([180, 255, 255])

image_files = sorted(input_dir.glob("*.png")) + sorted(input_dir.glob("*.jpg"))

if not image_files:
    print(f"No image files found in {input_dir}. Please place images there before running this step.")
else:
    print(f"Found {len(image_files)} image(s) in {input_dir}.")
    for img_path in image_files:
        # Read the image using OpenCV (or PIL then convert to numpy for cv2)
        img_pil = Image.open(img_path).convert("RGB")
        img_cv = np.array(img_pil)
        img_cv = cv2.cvtColor(img_cv, cv2.COLOR_RGB2BGR) # Convert from PIL's RGB to OpenCV's BGR

        # Convert BGR to HSV
        hsv = cv2.cvtColor(img_cv, cv2.COLOR_BGR2HSV)

        # Create masks for the two red ranges
        mask1 = cv2.inRange(hsv, lower_red_1, upper_red_1)
        mask2 = cv2.inRange(hsv, lower_red_2, upper_red_2)

        # Combine the masks
        full_mask = mask1 + mask2

        # Save the mask
        mask_filename = img_path.name # Keep original filename
        mask_save_path = output_dir / mask_filename
        cv2.imwrite(str(mask_save_path), full_mask)
        print(f"Saved mask for {img_path.name} to {mask_save_path}")

print("Mask generation complete.")

Found 20 image(s) in data_kontext/red.
Saved mask for im (1).png to data_kontext/masks/im (1).png
Saved mask for im (10).png to data_kontext/masks/im (10).png
Saved mask for im (11).png to data_kontext/masks/im (11).png
Saved mask for im (12).png to data_kontext/masks/im (12).png
Saved mask for im (13).png to data_kontext/masks/im (13).png
Saved mask for im (14).png to data_kontext/masks/im (14).png
Saved mask for im (15).png to data_kontext/masks/im (15).png
Saved mask for im (16).png to data_kontext/masks/im (16).png
Saved mask for im (17).png to data_kontext/masks/im (17).png
Saved mask for im (18).png to data_kontext/masks/im (18).png
Saved mask for im (19).png to data_kontext/masks/im (19).png
Saved mask for im (2).png to data_kontext/masks/im (2).png
Saved mask for im (20).png to data_kontext/masks/im (20).png
Saved mask for im (3).png to data_kontext/masks/im (3).png
Saved mask for im (4).png to data_kontext/masks/im (4).png
Saved mask for im (5).png to data_kontext/masks/im (5)

In [7]:
#@title Depth
DEPTH_MODEL_ID = "Intel/dpt-hybrid-midas" #@param {type: "string"}

depth_image_processor = DPTImageProcessor.from_pretrained(DEPTH_MODEL_ID)
depth_model = DPTForDepthEstimation.from_pretrained(DEPTH_MODEL_ID)

# Move the model to the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
depth_model.to(device)

print(f"Depth estimation model '{DEPTH_MODEL_ID}' loaded and moved to {device}.")

input_dir = os.path.join(DATA_ROOT, "context")
output_dir = os.path.join(DATA_ROOT, "depth")

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

image_files = sorted([f for f in os.listdir(input_dir) if f.endswith(('.png', '.jpg', '.jpeg'))])

if not image_files:
    print(f"No image files found in {input_dir}. Please place images there before running this step.")
else:
    print(f"Found {len(image_files)} image(s) in {input_dir}.")
    for img_filename in image_files:
        img_path = os.path.join(input_dir, img_filename)
        image = Image.open(img_path).convert("RGB")

        # Prepare image for the model
        inputs = depth_image_processor(images=image, return_tensors="pt").to(device)

        with torch.no_grad():
            # Predict depth map
            outputs = depth_model(**inputs)
            predicted_depth = outputs.predicted_depth

        # Interpolate to original size and convert to numpy
        prediction = F.interpolate(
            predicted_depth.unsqueeze(1),
            size=image.size[::-1], # PIL image size is (width, height), interpolate expects (height, width)
            mode="bicubic",
            align_corners=False,
        )
        output = prediction.squeeze().cpu().numpy()

        # Normalize to 0-255 for saving as an image
        formatted = (output * 255 / np.max(output)).astype('uint8')
        depth_map_image = Image.fromarray(formatted)

        # Save the depth map
        depth_map_filename = os.path.splitext(img_filename)[0] + ".png"
        depth_map_save_path = os.path.join(output_dir, depth_map_filename)
        depth_map_image.save(depth_map_save_path)
        print(f"Saved depth map for {img_filename} to {depth_map_save_path}")

print("Depth map generation complete.")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/382 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/490M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/490M [00:00<?, ?B/s]

Depth estimation model 'Intel/dpt-hybrid-midas' loaded and moved to cuda.
Found 20 image(s) in data_kontext/context.
Saved depth map for im (1) (1).png to data_kontext/depth/im (1) (1).png
Saved depth map for im (10) (1).png to data_kontext/depth/im (10) (1).png
Saved depth map for im (11) (1).png to data_kontext/depth/im (11) (1).png
Saved depth map for im (12) (1).png to data_kontext/depth/im (12) (1).png
Saved depth map for im (13) (1).png to data_kontext/depth/im (13) (1).png
Saved depth map for im (14) (1).png to data_kontext/depth/im (14) (1).png
Saved depth map for im (15) (1).png to data_kontext/depth/im (15) (1).png
Saved depth map for im (16) (1).png to data_kontext/depth/im (16) (1).png
Saved depth map for im (17) (1).png to data_kontext/depth/im (17) (1).png
Saved depth map for im (18) (1).png to data_kontext/depth/im (18) (1).png
Saved depth map for im (19) (1).png to data_kontext/depth/im (19) (1).png
Saved depth map for im (2) (1).png to data_kontext/depth/im (2) (1).png

In [8]:
#@title Create prompts
def create_prompts_file(prompt_text: str):
    """
    Creates a prompts.txt file by repeating the given prompt_text
    for each image found in DATA_ROOT/images.
    """
    images_dir = Path(DATA_ROOT + "/context")
    image_files = sorted(images_dir.glob("*.png")) + sorted(images_dir.glob("*.jpg"))
    image_count = len(image_files)

    if image_count == 0:
        print(f"No image files found in {images_dir}. Cannot create prompts.txt.")
        return

    prompts_file_path = Path(DATA_ROOT + "/prompts.txt")
    with open(prompts_file_path, "w", encoding="utf-8") as f:
        for _ in range(image_count):
            f.write(prompt_text + "\n")
    print(f"Successfully created {prompts_file_path} with {image_count} prompts.")

print("Function `create_prompts_file` defined.")
print()

sample_prompt = "Replace the object with lamp named artemide tizio, preserving the rest of the scene unchanged" #@param {type: "string"}
create_prompts_file(sample_prompt)

# Verify the content of prompts.txt (optional, for debugging)
with open(Path(DATA_ROOT + "/prompts.txt"), "r", encoding="utf-8") as f:
    print("Content of prompts.txt:")
    for line in f:
        print(line.strip())

Function `create_prompts_file` defined.

Successfully created data_kontext/prompts.txt with 20 prompts.
Content of prompts.txt:
Replace the object with lamp named artemide tizio, preserving the rest of the scene unchanged
Replace the object with lamp named artemide tizio, preserving the rest of the scene unchanged
Replace the object with lamp named artemide tizio, preserving the rest of the scene unchanged
Replace the object with lamp named artemide tizio, preserving the rest of the scene unchanged
Replace the object with lamp named artemide tizio, preserving the rest of the scene unchanged
Replace the object with lamp named artemide tizio, preserving the rest of the scene unchanged
Replace the object with lamp named artemide tizio, preserving the rest of the scene unchanged
Replace the object with lamp named artemide tizio, preserving the rest of the scene unchanged
Replace the object with lamp named artemide tizio, preserving the rest of the scene unchanged
Replace the object with la

In [9]:
#@title Service funcs
#@markdown Утилиты: нормализация, латенты, 2×2 packing, img_ids
def normalize_01_to_m11(img: torch.Tensor) -> torch.Tensor:
    # вход ToTensor() ∈ [0,1] -> [-1,1]
    return img * 2.0 - 1.0

# def to_latents(img_bchw: torch.Tensor) -> torch.Tensor:
#     # img_bchw в [-1,1]; кодируем и масштабируем как в пайплайне
#     return vae.encode(img_bchw).latent_dist.sample() * vae.config.scaling_factor

def to_latents(img_bchw: torch.Tensor) -> torch.Tensor:
    # img_bchw в [-1,1]; VAE ожидает тот же dtype, что и его веса (bf16/fp16/fp32)
    x = img_bchw.to(device=device, dtype=vae.dtype)
    return vae.encode(x).latent_dist.sample() * vae.config.scaling_factor

def pack_like_pipeline(lat: torch.Tensor):
    # [B,C,H,W] -> [B,(H/2*W/2), C*4] — 2×2 packing, как в Kontext
    B,C,H,W = lat.shape
    x = lat.view(B, C, H//2, 2, W//2, 2).permute(0,2,4,1,3,5).contiguous()
    x = x.view(B, (H//2)*(W//2), C*4)
    return x, (H//2, W//2)

def img_ids_grid(batch, h_tok, w_tok, mark, dtype):
    # img_ids: (тип, row, col); тип: 0.0=target, 1.0=context, 2.0=region
    g = torch.zeros(h_tok, w_tok, 3, device=device, dtype=dtype)
    g[...,1] = torch.arange(h_tok, device=device, dtype=dtype)[:,None]
    g[...,2] = torch.arange(w_tok, device=device, dtype=dtype)[None,:]
    g[...,0] = mark
    return g.view(1,-1,3).repeat(batch,1,1)

In [10]:
#@title Dataset class
##@markdown Датасет: data_kontext/{context,target,masks,prompts.txt}
#-----Old one------
class KontextPairs(Dataset):
    def __init__(self, root: str, resolution=1024, ip_resolution=224):
        self.root = Path(root)
        self.ctx = sorted((self.root/"context").glob("*"))
        self.tgt = sorted((self.root/"target").glob("*"))
        self.msk = sorted((self.root/"masks").glob("*"))
        self.depth= sorted((self.root/"depth").glob("*")) # Changed 'depths' to 'depth'
        with open(self.root/"prompts.txt","r",encoding="utf-8") as f:
            self.prompts = [l.strip() for l in f]

        assert len(self.ctx)==len(self.tgt), "target must match context count"
        assert len(self.ctx)==len(self.prompts), "prompts must match context count"
        assert len(self.depth)==len(self.ctx), "depths must match context count"
        assert len(self.msk)==len(self.ctx), "masks must match context count"

        self.resolution = resolution
        self.tfm = transforms.Compose([
            transforms.Resize((resolution, resolution), interpolation=transforms.InterpolationMode.BICUBIC),
            transforms.ToTensor(),  # [0,1]
        ])
        self.mtfm = transforms.Compose([
            transforms.Resize((resolution, resolution), interpolation=transforms.InterpolationMode.NEAREST),
            transforms.ToTensor(),  # [0,1]
        ])

        self.dtf= transforms.Compose((ip_resolution, ip_resolution))


    def __len__(self): return len(self.ctx)
    def __getitem__(self, i):
        c = Image.open(self.ctx[i]).convert("RGB")
        t = Image.open(self.tgt[i]).convert("RGB")
        d = Image.open(self.depth[i]).convert("RGB")
        m = Image.open(self.msk[i]).convert("L") if len(self.msk)==len(self.ctx) else Image.new("L", (self.resolution,self.resolution), 255)

        return {
            "context": self.tfm(c),      # [3,H,W] ∈ [0,1]
            "target":  self.tfm(t),
            "mask":    (self.mtfm(m) > 0.5).float(),  # бинарная
            "depth":   pil_to_tensor(d),
            "prompt":  self.prompts[i],
        }

In [11]:
#@title MaskRegionToken
#@markdown обучаемая проекция маски в 1 токен
#------Way one-----
class MaskRegionToken(nn.Module):
    """
    Маска на латентной сетке H'×W' -> [B,1,Cin] через один Linear.
    Обучается вместе с LoRA.
    """
    def __init__(self, h_tok: int, w_tok: int, c_in: int):
        super().__init__()
        self.proj = nn.Linear(h_tok*w_tok, c_in, bias=False)
    def forward(self, m_small: torch.Tensor):  # [B,1,H',W']
        b,_,h,w = m_small.shape
        v = m_small.view(b, -1)        # [B, H'*W']
        return self.proj(v).unsqueeze(1)  # [B,1,Cin]


# #-------Way two-------
# class MaskRegionToken(nn.Module):
#     """
#     Проекция карты маски в один «регион-токен» (1×C), согласованный по размерности с patch-токенами Flux.
#     """
#     def __init__(self, Htok, Wtok, C):
#         super().__init__()
#         self.proj = nn.Sequential(
#             nn.Conv2d(1, C, 3, padding=1),
#             nn.AdaptiveAvgPool2d((1,1)),
#             nn.Flatten(),
#         )
#         nn.init.kaiming_normal_(self.proj[0].weight, a=math.sqrt(5))
#         nn.init.zeros_(self.proj[0].bias)

#     def forward(self, mask):  # mask: [B,1,H,W] в [0,1]
#         return self.proj(mask)  # [B, C]


In [12]:
#@title Loss func

#---------Simple region loss----------
def kontext_dual_region_loss(
    pred_v: torch.Tensor,      # [B, N, C]  — предсказанная скорость модели для токенов цели
    tgt_tok: torch.Tensor,     # [B, N, C]  — токены таргет-латентов (упакованные 2x2)
    ctx_tok: torch.Tensor,     # [B, N, C]  — токены контекст-латентов (упакованные 2x2)
    noise: torch.Tensor,       # [B, N, C]  — использованный шум ε
    mask_small: torch.Tensor,  # [B, 1, H', W'] — бинарная/в [0,1] маска на латентной сетке
    h_tok: int, w_tok: int,    # размеры токен-сетки (H', W'), чтобы привести маску
    w_in: float = 1.0,         # вес лосса внутри маски (match target)
    w_out: float = 1.0,        # вес лосса вне маски (preserve context)
    eps: float = 1e-6,
):
    # 1) Маска -> токен-веса [B, N, 1]
    w = F.interpolate(mask_small, size=(h_tok, w_tok), mode="nearest")      # [B,1,H',W']
    w = w.flatten(2).transpose(1, 2)                                        # [B,N,1]
    w_comp = 1.0 - w                                                        # вне маски

    # 2) Внутримасочный лосс по скорости (flow matching):  v_target = ε - x_target
    v_tgt = noise - tgt_tok                                                 # [B,N,C]
    l_in  = ((pred_v - v_tgt)**2 * w).sum() / (w.sum() * pred_v.shape[-1] + eps)

    # 3) Внемасочный «identity»-штраф по реконструированным латентам:
    #    x_pred = ε - v_pred  должен совпадать с контекстом вне маски
    x_pred = noise - pred_v
    l_out  = ((x_pred - ctx_tok)**2 * w_comp).sum() / (w_comp.sum() * pred_v.shape[-1] + eps)

    return w_in * l_in + w_out * l_out


#---------L2 region loss----------
def collect_lora_named_params(peft_wrapped_module):
    # Например: transformer.named_parameters()
    return [(n, p) for n, p in peft_wrapped_module.named_parameters() if p.requires_grad]


def fm_masked_dual_loss_with_l2(
    pred_v,          # [B, N, C]  предсказанная скорость для токенов цели
    tgt_tok,         # [B, N, C]  токены таргета (после 2×2 packing)
    ctx_tok,         # [B, N, C]  токены контекста (после 2×2 packing)
    noise,           # [B, N, C]  использованный ε на текущем шаге
    mask_small,      # [B, 1, H', W'] маска на латентной сетке
    h_tok, w_tok,    # размеры токен-сетки (H', W')
    # веса задач
    w_in=1.0,        # вес совпадения с таргетом внутри маски (по v)
    w_out=1.0,       # вес сохранения контекста вне маски (по x = ε - v)
    # L2-регуляризация
    l2_region_lambda=1e-4,  # коэффициент L2 на проектор region_token
    l2_lora_lambda=1e-4,    # коэффициент L2 на LoRA-параметры
    region_token_module=None,   # nn.Module с проекцией маски (обязателен для L2 на region)
    lora_named_params=None,     # список (name,param) для LoRA-слоёв (см. ниже хелпер)
    eps=1e-6,
):
    # --- 1) Маска -> токен-веса [B, N, 1]
    w = F.interpolate(mask_small, size=(h_tok, w_tok), mode="nearest").flatten(2).transpose(1, 2)
    w_comp = 1.0 - w

    # --- 2) Внутримасочный лосс по скорости (flow matching): v_tgt = ε − x_tgt
    v_tgt = noise - tgt_tok
    l_in = ((pred_v - v_tgt)**2 * w).sum() / (w.sum() * pred_v.shape[-1] + eps)

    # --- 3) Внемасочный "identity": x_pred = ε − v_pred должен совпадать с контекстом
    x_pred = noise - pred_v
    l_out = ((x_pred - ctx_tok)**2 * w_comp).sum() / (w_comp.sum() * pred_v.shape[-1] + eps)

    # --- 4) L2-регуляризация (не дублируйте с weight_decay в AdamW)
    l2_reg = pred_v.new_tensor(0.0)
    if region_token_module is not None:
        for p in region_token_module.parameters():
            if p.requires_grad:
                l2_reg = l2_reg + p.pow(2).sum()
    l2_lora = pred_v.new_tensor(0.0)
    if lora_named_params is not None:
        for name, p in lora_named_params:
            # обычно LoRA-параметры называются lora_A / lora_B в PEFT
            if p.requires_grad and ("lora_" in name):
                l2_lora = l2_lora + p.pow(2).sum()

    return w_in * l_in + w_out * l_out + l2_region_lambda * l2_reg + l2_lora_lambda * l2_lora


##Run

In [13]:
#@title 1) Загрузка Flux Kontext пайплайна (как в доках)
MODEL_ID = "black-forest-labs/FLUX.1-Kontext-dev"  # может требовать токен HF
pipe = FluxKontextPipeline.from_pretrained(MODEL_ID, torch_dtype=dtype).to(device)  #
vae, transformer, scheduler = pipe.vae, pipe.transformer, pipe.scheduler

model_index.json:   0%|          | 0.00/688 [00:00<?, ?B/s]

Fetching 23 files:   0%|          | 0/23 [00:00<?, ?it/s]

text_encoder/model.safetensors:   0%|          | 0.00/246M [00:00<?, ?B/s]

text_encoder_2/model-00001-of-00002.safe(…):   0%|          | 0.00/4.99G [00:00<?, ?B/s]

scheduler_config.json:   0%|          | 0.00/486 [00:00<?, ?B/s]

...

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
#@title 2) PEFT LoRA: таргетинг attention-линеек (как принято для FLUX)

#@markdown VAE и текстовые энкодеры лучше заморозить явно, чтобы автоград не строил по ним граф и не держал лишние активации — это экономит память/время. Это стандарт в примерах Diffusers (DreamBooth/LoRA): vae.requires_grad_(False), text_encoder.requires_grad_(False) и т. п. \ (есть issues)


# Заморозим всё, кроме будущих адаптеров
# for m in (vae, pipe.text_encoder, pipe.text_encoder_2, transformer):
#     m.requires_grad_(False)

#the same without loop
vae.requires_grad_(False)
pipe.text_encoder.requires_grad_(False)
pipe.text_encoder_2.requires_grad_(False)


target_modules = ["attn.to_q","attn.to_k","attn.to_v","attn.to_out.0"]
lora_cfg = LoraConfig(
    r=16,
    lora_alpha=16,          # держим == r для совместимости
    lora_dropout=0.0,
    bias="none",
    target_modules=target_modules,
    init_lora_weights="gaussian",
)
transformer.add_adapter(lora_cfg, adapter_name="mask_lora")   # PeftAdapterMixin
transformer.set_adapter("mask_lora")
print("LoRA adapter attached.")

LoRA adapter attached.


In [15]:
#@title Load IPAdapter
pipe.load_ip_adapter(
    "XLabs-AI/flux-ip-adapter-v2",
    weight_name="ip_adapter.safetensors",
    image_encoder_pretrained_model_name_or_path="openai/clip-vit-large-patch14"
)

IP_SCALE=0.6 #@param {type: "number"}
pipe.set_ip_adapter_scale(IP_SCALE)

ip_adapter.safetensors:   0%|          | 0.00/1.06G [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

In [16]:
#@title Load Data / Setup Settings

# --- 6) Подготовка данных/модулей ---
# см. структуру выше



#@markdown
#
#@markdown 1) prepare and load data
ds = KontextPairs(DATA_ROOT, resolution=RES)
dl = DataLoader(ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=0, drop_last=True)

#@markdown 2) Узнаём латентную сетку и создаём region_token под Cin модели
with torch.no_grad():
    tmp = next(iter(dl))
    lat = to_latents(normalize_01_to_m11(tmp["context"].to(device))).to(dtype)
    _, (Htok, Wtok) = pack_like_pipeline(lat)
region_token = MaskRegionToken(Htok, Wtok, transformer.config.in_channels).to(device)

#@markdown 3) Оптимизатор: PEFT отмечает адаптерные веса как trainable; добавляем region_token
opt = torch.optim.AdamW(
    list(region_token.parameters()) + [p for p in transformer.parameters() if p.requires_grad],
    lr=LR, betas=(0.9,0.999), weight_decay=1e-2
)

In [18]:
#@title Run
transformer.train()
vae.eval()
pipe.text_encoder.eval()
pipe.text_encoder_2.eval()

#-----For L2 region loss-----
lora_named = collect_lora_named_params(transformer)  # transformer с add_adapter/set_adapter

global_step = 0

for epoch in tqdm(range(EPOCHS)):
    for batch in tqdm(dl):
        global_step += 1

        # (a) Данные
        ctx = normalize_01_to_m11(batch["context"].to(device))  # [-1,1]
        tgt = normalize_01_to_m11(batch["target" ].to(device))
        msk = batch["mask"].to(device)
        #pipe.prepare_ip_adapter_image_embeds will prepare and send depth to device by itself.
        depth_pil=to_pil_image(batch["depth"][0])
        prompts = batch["prompt"]

        # (b) Латенты и 2×2 packing
        ctx_lat = to_latents(ctx).to(dtype)  # [B,C,H',W']
        tgt_lat = to_latents(tgt).to(dtype)
        tgt_tok,(Htok,Wtok) = pack_like_pipeline(tgt_lat)  # [B,N,Cin]
        ctx_tok,_           = pack_like_pipeline(ctx_lat)
        B,N,Cin = tgt_tok.shape

        # (c) Flow-matching таргет (упрощённо): target = noise - latents; σ из шедулера

        # idx   = torch.randint(0, len(scheduler.sigmas), (B,), device=device)
        # sigma = scheduler.sigmas[idx].view(B,1,1).to(dtype=tgt_tok.dtype)   #



        # sigmas = scheduler.sigmas.to(device=tgt_tok.device, dtype=tgt_tok.dtype)
        # idx    = torch.randint(0, sigmas.shape[0], (B,), device=tgt_tok.device)
        # sigma  = sigmas.index_select(0, idx).view(B, 1, 1)

        # noise = torch.randn_like(tgt_tok)
        # noised= tgt_tok + sigma * noise
        # target= (noise - tgt_tok).detach()



        # sigmas и индексы — на том же устройстве/типе, что и токены
        sigmas = scheduler.sigmas.to(device=tgt_tok.device, dtype=tgt_tok.dtype)
        idx    = torch.randint(sigmas.numel(), (B,), device=tgt_tok.device)
        # timestep должен быть 1D [B]
        timestep = sigmas.index_select(0, idx)  # [B]
        noise    = torch.randn_like(tgt_tok)
        noised   = tgt_tok + timestep.view(B,1,1) * noise
        target   = (noise - tgt_tok).detach()



        # (d) region_token из маски (на латентной сетке)
        m_small = F.interpolate(msk, size=(Htok, Wtok), mode="nearest")  # [B,1,H',W']
        r_tok   = region_token(m_small).to(dtype)                        # [B,1,Cin]

        # (e) Визуальная последовательность и img_ids
        hidden = torch.cat([noised, ctx_tok, r_tok], dim=1)              # [B,2N+1,Cin]
        ids_t  = img_ids_grid(B, Htok, Wtok, mark=0.0, dtype=hidden.dtype)
        ids_c  = img_ids_grid(B, Htok, Wtok, mark=1.0, dtype=hidden.dtype)
        # центр маски по токен-сетке
        m2 = m_small.squeeze(1)                               # [B,H',W']
        wy = (m2.sum(-1) + 1e-6); wx = (m2.sum(-2) + 1e-6)
        ys = (m2 * torch.arange(Htok, device=device)[None,:,None]).sum(dim=(1,2)) / wy.sum(-1)
        xs = (m2 * torch.arange(Wtok, device=device)[None,None,:]).sum(dim=(1,2)) / wx.sum(-1)
        ids_r  = torch.stack([torch.full_like(ys,2.0), ys, xs], dim=-1).view(B,1,3).to(dtype=hidden.dtype)
        img_ids= torch.cat([ids_t, ids_c, ids_r], dim=1)                 # [B,2N+1,3]

        # (f) Текстовые эмбеддинги как в пайплайне: T5 seq + CLIP pooled (encode_prompt)

        # prompt_embeds, pooled_embeds, txt_ids, *_ = pipe.encode_prompt(
        #     prompts, device=device, num_images_per_prompt=1, do_classifier_free_guidance=False
        # )  #

        # Для Kontext нужно передать два текста: CLIP (prompt) и T5 (prompt_2).
        # Если у вас один общий текст, безопасно продублировать:
        prompt_embeds, pooled_embeds, txt_ids = pipe.encode_prompt(
            prompt=prompts,            # CLIP-подсказка (pooled)
            prompt_2=prompts,          # T5-подсказка (seq)
            device=device,
            num_images_per_prompt=1,
            max_sequence_length=256,   # опционально (<=512)
        )


        guidance_tensor = torch.tensor(1.0, device=hidden.device, dtype=hidden.dtype)

        # # (g) Прямой проход трансформера — предсказываем velocity для токенов цели
        # out = transformer(
        #     hidden_states=hidden,
        #     timestep=sigma.squeeze(-1).squeeze(-1),
        #     encoder_hidden_states=prompt_embeds.to(dtype),
        #     pooled_projections=pooled_embeds.to(dtype),
        #     img_ids=img_ids,
        #     txt_ids=txt_ids.to(dtype),
        #     guidance=guidance_tensor,#1.0,
        #     return_dict=False,
        # )[0][:, :N, :]  # берём только N токенов цели


        #-------------------IPAdapter---------------------
        ip_embeds = pipe.prepare_ip_adapter_image_embeds(
                                                          ip_adapter_image=depth_pil,     # можно список, если адаптеров несколько
                                                          ip_adapter_image_embeds=None,
                                                          device=device,
                                                          num_images_per_prompt=1,
                                                          #do_classifier_free_guidance=False,
                                                      )

        # 3) «Инъекция» в те же projection-слои, что использует пайплайн
        # for proj_layer, emb in zip(transformer.encoder_hid_proj.image_projection_layers, ip_embeds):
        #     proj_layer.image_embeds = emb
        #--------------------------------------------------

        #---------------------Generation-------------------
        out = transformer(
                    hidden_states=hidden,
        #            timestep=sigma.squeeze(-1).squeeze(-1),
                    timestep=timestep,               # 1D [B], без squeeze-танцев
                    encoder_hidden_states=prompt_embeds.to(dtype),
                    pooled_projections=pooled_embeds.to(dtype),
                    img_ids=img_ids,
        #            txt_ids=txt_ids.to(dtype),
        #            guidance=None,          # <— не передаём скаляр float
                    txt_ids=txt_ids,                  # оставляем dtype, как вернул encode_prompt
                    # guidance не передаём вовсе (по умолчанию None). Если нужно явно:
                    guidance=timestep.new_ones(B),  # тоже 1D [B]
                    return_dict=False,
                    joint_attention_kwargs={"ip_adapter_image_embeds": ip_embeds},
                )[0][:, :N, :]



        # # (h) Лосс (минимальный). При желании можно взвесить по маске на токен-сетке
        # loss = F.mse_loss(out.float(), target.float())

        #------Simple region loss-------
        # loss = kontext_dual_region_loss(
        #                                 pred_v=out, tgt_tok=tgt_tok, ctx_tok=ctx_tok, noise=noise,
        #                                 mask_small=m_small, h_tok=Htok, w_tok=Wtok,
        #                                 w_in=1.0, w_out=1.0
        #                                 )


        #------L2 region loss----------
        loss = fm_masked_dual_loss_with_l2(
            pred_v=out, tgt_tok=tgt_tok, ctx_tok=ctx_tok, noise=noise,
            mask_small=m_small, h_tok=Htok, w_tok=Wtok,
            w_in=1.0, w_out=1.0,
            l2_region_lambda=1e-4,  # подберите по валидации
            l2_lora_lambda=1e-4,    # подберите по валидации
            region_token_module=region_token,
            lora_named_params=lora_named,
        )


        opt.zero_grad(set_to_none=True)
        loss.backward()
        opt.step()

        if global_step % 25 == 0:
            print(f"step {global_step} | loss {loss.item():.5f}")

        # # (i) Валидационный семпл (пайплайн; маску НЕ передаём — она только локализует обучение)
        # if global_step % SAMPLE_EVERY == 0:
        #     clear_output(wait=True)
        #     transformer.eval()
        #     with torch.autocast(device.type, dtype=dtype):
        #         vis = transforms.ToPILImage()(batch["context"][0].cpu())
        #         img = pipe(image=vis, prompt=prompts[0], num_inference_steps=24, guidance_scale=2.5).images[0]
        #     display(img)
        #     transformer.train()


print("Training done.")

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

step 25 | loss 4.28543
Training done.


In [ ]:
#@title Save result
#@markdown Сохранение адаптера штатным методом (PeftAdapterMixin)
Path("result").mkdir(parents=True, exist_ok=True)
transformer.save_lora_adapter(save_directory="result", adapter_name="mask_lora")
print("LoRA adapter saved to ./result")


old_file_path = "./result/pytorch_lora_weights.safetensors"
new_file_path = "./result/masked_lora.safetensors"

if os.path.exists(old_file_path):
    os.rename(old_file_path, new_file_path)
    print(f"\nRenamed '{old_file_path}' to '{new_file_path}'")
    print("Contents after renaming:")
else:
    print(f"\nFile '{old_file_path}' not found. Cannot rename.")


#@markdown Сохранение region_token кастомным методом
state = {f"region_token.{k}": v.detach().cpu() for k, v in region_token.state_dict().items()}
save_file(state, "result/region_token.safetensors")
print("Region token saved.")
print()

!ls -F result

LoRA adapter saved to ./result

Renamed './result/pytorch_lora_weights.safetensors' to './result/masked_lora.safetensors'
Contents after renaming:
Region token saved.

masked_lora.safetensors  region_token.safetensors


In [ ]:
#@title Load region_token example
#from safetensors.torch import load_file
ckpt = load_file("result/region_token.safetensors")
region_token.load_state_dict({k.split("region_token.",1)[1]: v for k, v in ckpt.items()})
region_token.to(device)

MaskRegionToken(
  (proj): Linear(in_features=1024, out_features=64, bias=False)
)